# Notebook 4: Spatial Statistics with `squidpy`

**Instructor:** Anthony Christidis 
**Time:** 45 minutes

---

Welcome to our deep dive into spatial statistics! In the previous notebook, we processed our high-resolution **Xenium data** and identified cell clusters. Now, we will use `squidpy` to ask advanced biological questions about the spatial organization of those cells and genes.

**Goals:**
1.  Identify spatially organized genes using **Moran's I**.
2.  Analyze cell community structures using **neighborhood enrichment**.
3.  Quantify how often different cell types appear near each other using **co-occurrence** analysis.

### Setup and Data Loading

We will start by loading the **fully processed and clustered Xenium `AnnData` object** that we saved at the end of the previous notebook. This allows us to jump straight into the spatial analysis.

In [ ]:
%load_ext jupyter_black

import anndata as ad
import squidpy as sq
import scanpy as sc

import warnings
warnings.filterwarnings("ignore")

# Load the processed AnnData object
adata_xenium = ad.read_h5ad("../data/processed/adata_xenium_processed.h5ad")
print("Processed Xenium data loaded.")

In [ ]:
adata_xenium

### Building the Spatial Graph

All of `squidpy`'s tools rely on a **spatial neighborhood graph**. This graph connects cells that are physically close to each other in the tissue. We build it once from the cell coordinates, and then we can use it for many different analyses.

Note: The coordinates were not saved in the `.h5ad` file, so we need to re-extract them from the original `SpatialData` object.

In [ ]:
import spatialdata as sd

# Load the original sdata object just to get the coordinates
sdata_xenium = sd.read_zarr("../data/xenium_lung_cancer_subset.zarr")

# Use .loc to safely get the coordinates for our processed cells
adata_xenium.obsm['spatial'] = sdata_xenium.shapes['cell_circles'].loc[adata_xenium.obs_names].centroid.apply(lambda p: [p.x, p.y]).to_numpy()

# Now, build the graph
sq.gr.spatial_neighbors(adata_xenium, coord_type="generic", delaunay=True)

Let's visualize the clusters we found in the previous notebook to confirm everything is correct.

In [ ]:
sc.pl.spatial(adata_xenium, color="clusters", spot_size=30, frameon=False, title="Xenium Cell Clusters")

### Analysis 1: Spatially Variable Genes (Moran's I)

**Biological Question:** Which genes' expression levels are not random, but are clustered in specific tissue regions? This helps identify genes that define anatomical structures.

In [ ]:
# Calculate Moran's I using the graph we just built
# We'll test only the highly variable genes we found earlier for speed.
sq.gr.spatial_autocorr(adata_xenium, mode="moran", genes=adata_xenium.var.highly_variable, n_perms=100, n_jobs=4)

# Show the top results, sorted by the 'I' score
adata_xenium.uns["moranI"].sort_values(by="I", ascending=False).head()

Now we can visualize the expression of the top-scoring gene using `scanpy`'s robust spatial plotting function.

In [ ]:
top_gene = adata_xenium.uns["moranI"].sort_values(by="I", ascending=False).index[0]
sc.pl.spatial(adata_xenium, color=top_gene, spot_size=30, frameon=False, title=f"{top_gene} Expression")

### Analysis 2: Neighborhood Enrichment

Now let's analyze the organization of our cell clusters.

**Biological Question:** Which cell types (Leiden clusters) are found together more or less often than expected by chance? This helps us find potential cell-cell communication hubs or excluded cell populations.

In [ ]:
sq.gr.nhood_enrichment(adata_xenium, cluster_key="clusters")

We visualize this as a heatmap. Bright yellow indicates that two clusters are often neighbors (co-enriched), while dark purple indicates they tend to avoid each other (depleted).

In [ ]:
sq.pl.nhood_enrichment(adata_xenium, cluster_key="clusters", method="ward", cmap="viridis")

This plot gives us powerful, quantifiable insights into the tissue's "social network." For example, we might discover a strong interaction between immune cells and tumor cells based on a high enrichment score.

### Workshop Conclusion
Congratulations! You have now completed a full spatial analysis workflow, from QC and clustering to advanced spatial statistics. You are now equipped with the core `scverse` tools to begin exploring your own spatial datasets.